##Main dataset
The main dataset consists of the following columns: company, title, link, publish_date, label, score, type, and probability.

- **company**: Company name
- **title**: News title
- **link**: Link to the news article
- **publish_date**: Date when the news was published
- **label**: Indicates whether the news is 'positive' or 'negative'
- **score**: Weight for the label
- **type**: Categorizes the news as 'government', 'social', 'environment', or 'neutral'
- **probability**: Probability weight for the label

The main idea is to generate new datasets for each company. Each dataset should consist of stock market prices, percentage of positive environment news, percentage of negative environment news, percentage of positive social news, percentage of negative social news, percentage of positive government news, percentage of negative government news, percentage of positive neutral news, and percentage of negative neutral news for each day from July 2023 to August 2024. It is possible that for some days there won't be any data.

The companies are: Pfizer, Procter and Gamble, Tencent, Samsung, Tesla, Visa, Intel, Walmart.

These datasets will be merged later and used as input to the XGBoost model.

---

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Dataset] Zero Shot/news_titles-FINAL.csv")

In [5]:
df.head()

,company,title,link,publish_date,label,score,type,probability
0,Oracle,Oracle and NVIDIA to Deliver Sovereign AI Worl...,https://news.google.com/read/CBMibkFVX3lxTE5SV...,2024-03-18T07:00:00.000Z,POSITIVE,0.996672,government,0.515955
1,Oracle,Introducing Oracle Health Clinical Digital Ass...,https://news.google.com/read/CBMifEFVX3lxTFBGR...,2024-06-22T01:28:20.000Z,POSITIVE,0.990025,social,0.357664
2,Oracle,"Optimize your Oracle estate, reduce costs and ...",https://news.google.com/read/CBMiW0FVX3lxTE5Hc...,2024-08-07T21:22:53.000Z,POSITIVE,0.993441,social,0.348627
3,Oracle,OpenAI Selects Oracle Cloud Infrastructure to ...,https://news.google.com/read/CBMi0gFBVV95cUxNN...,2024-06-11T07:00:00.000Z,POSITIVE,0.868924,environment,0.452918
4,Oracle,Microsoft and Oracle expand partnership to del...,https://news.google.com/read/CBMi9wFBVV95cUxOR...,2023-09-14T07:00:00.000Z,POSITIVE,0.990156,environment,0.415351


In [6]:
df.shape

(1218, 8)

In [7]:
df.columns

Index(['company', 'title', 'link', 'publish_date', 'label', 'score', 'type',
       'probability'],
      dtype='object')

In [8]:
unique_values_company = df['company'].unique()

In [9]:
print(unique_values_company)

['Oracle' 'Procter' 'Pfizer' 'Samsung' 'Shell' 'Tencent' 'Toyota' 'Tesla'
 'Unilever' 'UnitedHealth' 'Visa' 'Walmart' 'Intel']


We will drop the 'probability' and 'score' columns

In [10]:
df = df.drop(columns=['probability', 'score'])

In [11]:
df.columns

Index(['company', 'title', 'link', 'publish_date', 'label', 'type'], dtype='object')

In [12]:
# Convert the column to datetime format
df['publish_date'] = pd.to_datetime(df['publish_date'])

# Format the datetime column to 'DD/MM/YYYY'
df['date'] = df['publish_date'].dt.strftime('%d/%m/%Y')

In [13]:
df.columns

Index(['company', 'title', 'link', 'publish_date', 'label', 'type', 'date'], dtype='object')

In [14]:
df.head()

,company,title,link,publish_date,label,type,date
0,Oracle,Oracle and NVIDIA to Deliver Sovereign AI Worl...,https://news.google.com/read/CBMibkFVX3lxTE5SV...,2024-03-18 07:00:00+00:00,POSITIVE,government,18/03/2024
1,Oracle,Introducing Oracle Health Clinical Digital Ass...,https://news.google.com/read/CBMifEFVX3lxTFBGR...,2024-06-22 01:28:20+00:00,POSITIVE,social,22/06/2024
2,Oracle,"Optimize your Oracle estate, reduce costs and ...",https://news.google.com/read/CBMiW0FVX3lxTE5Hc...,2024-08-07 21:22:53+00:00,POSITIVE,social,07/08/2024
3,Oracle,OpenAI Selects Oracle Cloud Infrastructure to ...,https://news.google.com/read/CBMi0gFBVV95cUxNN...,2024-06-11 07:00:00+00:00,POSITIVE,environment,11/06/2024
4,Oracle,Microsoft and Oracle expand partnership to del...,https://news.google.com/read/CBMi9wFBVV95cUxOR...,2023-09-14 07:00:00+00:00,POSITIVE,environment,14/09/2023


In [15]:
df['publish_date'] = pd.to_datetime(df['publish_date'])

# Format the datetime column to 'YYYY-MM-DD' (this will remove the time and timezone)
df['Date'] = df['publish_date'].dt.date

In [16]:
df.head()

,company,title,link,publish_date,label,type,date,Date
0,Oracle,Oracle and NVIDIA to Deliver Sovereign AI Worl...,https://news.google.com/read/CBMibkFVX3lxTE5SV...,2024-03-18 07:00:00+00:00,POSITIVE,government,18/03/2024,2024-03-18
1,Oracle,Introducing Oracle Health Clinical Digital Ass...,https://news.google.com/read/CBMifEFVX3lxTFBGR...,2024-06-22 01:28:20+00:00,POSITIVE,social,22/06/2024,2024-06-22
2,Oracle,"Optimize your Oracle estate, reduce costs and ...",https://news.google.com/read/CBMiW0FVX3lxTE5Hc...,2024-08-07 21:22:53+00:00,POSITIVE,social,07/08/2024,2024-08-07
3,Oracle,OpenAI Selects Oracle Cloud Infrastructure to ...,https://news.google.com/read/CBMi0gFBVV95cUxNN...,2024-06-11 07:00:00+00:00,POSITIVE,environment,11/06/2024,2024-06-11
4,Oracle,Microsoft and Oracle expand partnership to del...,https://news.google.com/read/CBMi9wFBVV95cUxOR...,2023-09-14 07:00:00+00:00,POSITIVE,environment,14/09/2023,2023-09-14


In [17]:
df = df.drop(columns=['date'])

In [18]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Datasets]/main_dataset-final.csv")

##Pfizer stock prizes and news dataset

In [4]:
df_pfizer = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Datasets]/[Dataset] Stock prices/PFE.csv')

In [5]:
df_pfizer.head(4)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-08-28,36.439999,36.520000,36.029999,36.209999,34.193111,15101900
1,2023-08-29,36.259998,36.939999,36.000000,36.150002,34.136456,21076500
2,2023-08-30,36.180000,36.250000,35.820000,35.900002,33.900383,16833000
3,2023-08-31,35.939999,35.990002,35.380001,35.380001,33.409348,27919500


In [6]:
df_pfizer['Date'] = pd.to_datetime(df_pfizer['Date'], format='%Y-%m-%d').dt.date

In [23]:
pfizer_news = df[df['company'] == 'Pfizer']

In [24]:
pfizer_news.head()

,company,title,link,publish_date,label,type,Date
196,Pfizer,Pfizer’s LORBRENA® CROWN Study Shows Majority ...,https://news.google.com/read/CBMivAFBVV95cUxPR...,2024-05-31 07:00:00+00:00,POSITIVE,social,2024-05-31
197,Pfizer,Comparing the COVID-19 Vaccines: How Are They ...,https://news.google.com/read/CBMibkFVX3lxTE1GU...,2024-04-24 07:00:00+00:00,NEGATIVE,social,2024-04-24
198,Pfizer,Pfizer boosts bioreactor efficiency with AWS i...,https://news.google.com/read/CBMirwFBVV95cUxNV...,2023-10-12 07:00:00+00:00,POSITIVE,government,2023-10-12
199,Pfizer,Ginkgo Bioworks Announces Multi-Target RNA Dis...,https://news.google.com/read/CBMizwFBVV95cUxNa...,2023-09-27 07:00:00+00:00,POSITIVE,social,2023-09-27
200,Pfizer,Takeda and Pfizer Announce Four-Year Results f...,https://news.google.com/read/CBMivwFBVV95cUxNc...,2024-06-01 07:00:00+00:00,POSITIVE,government,2024-06-01


In [25]:
pfizer_news.columns

Index(['company', 'title', 'link', 'publish_date', 'label', 'type', 'Date'], dtype='object')

In [26]:
print(pfizer_news['type'].value_counts())

type
social         66
government     27
environment     4
neutral         1
Name: count, dtype: int64


In [27]:
print(pfizer_news['label'].value_counts())

label
POSITIVE    56
NEGATIVE    42
Name: count, dtype: int64


In [28]:
pfizer_df = df[df['company'] == 'Pfizer']

merged_df_pfizer = pd.merge(pfizer_df, df_pfizer, on=['Date'])

print(merged_df_pfizer.head())

  company                                              title  \
0  Pfizer  Pfizer’s LORBRENA® CROWN Study Shows Majority ...   
1  Pfizer  Pfizer sees lung cancer drug topping $1 billio...   
2  Pfizer  Comparing the COVID-19 Vaccines: How Are They ...   
3  Pfizer  Pfizer boosts bioreactor efficiency with AWS i...   
4  Pfizer  Cancer Survivors' 'Butterfly Club' Network Fos...   

                                                link  \
0  https://news.google.com/read/CBMivAFBVV95cUxPR...   
1  https://news.google.com/read/CBMi5wFBVV95cUxQM...   
2  https://news.google.com/read/CBMibkFVX3lxTE1GU...   
3  https://news.google.com/read/CBMirwFBVV95cUxNV...   
4  https://news.google.com/read/CBMirAFBVV95cUxOZ...   

               publish_date     label        type        Date       Open  \
0 2024-05-31 07:00:00+00:00  POSITIVE      social  2024-05-31  28.340000   
1 2024-05-31 07:00:00+00:00  POSITIVE      social  2024-05-31  28.340000   
2 2024-04-24 07:00:00+00:00  NEGATIVE      social 

In [30]:
merged_df_pfizer = merged_df_pfizer.drop(columns=['Volume', 'Adj Close'])

In [31]:
merged_df_pfizer.head()

,company,title,link,publish_date,label,type,Date,Open,High,Low,Close
0,Pfizer,Pfizer’s LORBRENA® CROWN Study Shows Majority ...,https://news.google.com/read/CBMivAFBVV95cUxPR...,2024-05-31 07:00:00+00:00,POSITIVE,social,2024-05-31,28.340000,28.750000,28.240000,28.660000
1,Pfizer,Pfizer sees lung cancer drug topping $1 billio...,https://news.google.com/read/CBMi5wFBVV95cUxQM...,2024-05-31 07:00:00+00:00,POSITIVE,social,2024-05-31,28.340000,28.750000,28.240000,28.660000
2,Pfizer,Comparing the COVID-19 Vaccines: How Are They ...,https://news.google.com/read/CBMibkFVX3lxTE1GU...,2024-04-24 07:00:00+00:00,NEGATIVE,social,2024-04-24,26.250000,26.340000,26.040001,26.270000
3,Pfizer,Pfizer boosts bioreactor efficiency with AWS i...,https://news.google.com/read/CBMirwFBVV95cUxNV...,2023-10-12 07:00:00+00:00,POSITIVE,government,2023-10-12,33.110001,33.220001,32.770000,32.919998
4,Pfizer,Cancer Survivors' 'Butterfly Club' Network Fos...,https://news.google.com/read/CBMirAFBVV95cUxOZ...,2023-10-12 13:30:14+00:00,POSITIVE,social,2023-10-12,33.110001,33.220001,32.770000,32.919998


In [32]:
from collections import Counter

# Group by 'date' and 'type', aggregate sentiment scores and count news types
grouped_data = df.groupby(['Date', 'type']).agg({
    'label': lambda x: Counter(x),
    'type': 'count'
}).rename(columns={'type': 'type_count'}).reset_index()

print(grouped_data)

           Date         type                           label  type_count
0    2023-08-22  environment                 {'NEGATIVE': 1}           1
1    2023-08-22   government                 {'NEGATIVE': 1}           1
2    2023-08-24   government  {'POSITIVE': 1, 'NEGATIVE': 1}           2
3    2023-08-24       social                 {'NEGATIVE': 1}           1
4    2023-08-28   government                 {'NEGATIVE': 1}           1
..          ...          ...                             ...         ...
573  2024-08-17       social  {'POSITIVE': 1, 'NEGATIVE': 3}           4
574  2024-08-18       social  {'POSITIVE': 2, 'NEGATIVE': 2}           4
575  2024-08-19  environment                 {'NEGATIVE': 1}           1
576  2024-08-19       social                 {'NEGATIVE': 2}           2
577  2024-08-20       social                 {'POSITIVE': 1}           1

[578 rows x 4 columns]


In [33]:
grouped_data.head(30)

,Date,type,label,type_count
0,2023-08-22,environment,{'NEGATIVE': 1},1
1,2023-08-22,government,{'NEGATIVE': 1},1
2,2023-08-24,government,"{'POSITIVE': 1, 'NEGATIVE': 1}",2
3,2023-08-24,social,{'NEGATIVE': 1},1
4,2023-08-28,government,{'NEGATIVE': 1},1
5,2023-08-28,social,{'NEGATIVE': 1},1
6,2023-08-29,environment,{'POSITIVE': 1},1
7,2023-08-29,government,{'POSITIVE': 1},1
8,2023-08-30,environment,{'NEGATIVE': 1},1
9,2023-08-30,neutral,{'NEGATIVE': 1},1


In [34]:
# Initialize new columns for percentages
grouped_data['negative_percentage'] = 0.0
grouped_data['positive_percentage'] = 0.0

# Iterate through the rows to calculate the percentages
for i, row in grouped_data.iterrows():
    sentiment_dict = row['label']

    # Get the counts of NEGATIVE and POSITIVE sentiments
    negative_count = sentiment_dict.get('NEGATIVE', 0)
    positive_count = sentiment_dict.get('POSITIVE', 0)
    total_count = row['type_count']

    # Calculate the percentages and assign to the new columns
    if total_count > 0:
        grouped_data.at[i, 'negative_percentage'] = (negative_count / total_count) * 100
        grouped_data.at[i, 'positive_percentage'] = (positive_count / total_count) * 100

# Display the updated DataFrame with the new columns
print(grouped_data[['Date', 'type', 'negative_percentage', 'positive_percentage']])


           Date         type  negative_percentage  positive_percentage
0    2023-08-22  environment                100.0                  0.0
1    2023-08-22   government                100.0                  0.0
2    2023-08-24   government                 50.0                 50.0
3    2023-08-24       social                100.0                  0.0
4    2023-08-28   government                100.0                  0.0
..          ...          ...                  ...                  ...
573  2024-08-17       social                 75.0                 25.0
574  2024-08-18       social                 50.0                 50.0
575  2024-08-19  environment                100.0                  0.0
576  2024-08-19       social                100.0                  0.0
577  2024-08-20       social                  0.0                100.0

[578 rows x 4 columns]


In [35]:
# Pivot the DataFrame to get each type as separate columns for negative and positive percentages
pivot_df = grouped_data.pivot(index='Date', columns='type', values=['negative_percentage', 'positive_percentage'])

# Flatten the multi-level column names
pivot_df.columns = [f"{col[1]}_{col[0]}" for col in pivot_df.columns]

# Reset the index to bring 'Date' back as a column
pivot_df = pivot_df.reset_index()

# Display the resulting DataFrame
print(pivot_df.head())

         Date  environment_negative_percentage  \
0  2023-08-22                            100.0   
1  2023-08-24                              NaN   
2  2023-08-28                              NaN   
3  2023-08-29                              0.0   
4  2023-08-30                            100.0   

   government_negative_percentage  neutral_negative_percentage  \
0                           100.0                          NaN   
1                            50.0                          NaN   
2                           100.0                          NaN   
3                             0.0                          NaN   
4                             NaN                        100.0   

   social_negative_percentage  environment_positive_percentage  \
0                         NaN                              0.0   
1                       100.0                              NaN   
2                       100.0                              NaN   
3                         NaN         

In [36]:
pivot_df.head(30)

,Date,environment_negative_percentage,government_negative_percentage,neutral_negative_percentage,social_negative_percentage,environment_positive_percentage,government_positive_percentage,neutral_positive_percentage,social_positive_percentage
0,2023-08-22,100.000000,100.0,NaN,NaN,0.000000,0.0,NaN,NaN
1,2023-08-24,NaN,50.0,NaN,100.000000,NaN,50.0,NaN,0.000000
2,2023-08-28,NaN,100.0,NaN,100.000000,NaN,0.0,NaN,0.000000
3,2023-08-29,0.000000,0.0,NaN,NaN,100.000000,100.0,NaN,NaN
4,2023-08-30,100.000000,NaN,100.0,0.000000,0.000000,NaN,0.0,100.000000
5,2023-08-31,100.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
6,2023-09-01,66.666667,NaN,NaN,NaN,33.333333,NaN,NaN,NaN
7,2023-09-03,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,0.000000
8,2023-09-05,0.000000,NaN,NaN,0.000000,100.000000,NaN,NaN,100.000000
9,2023-09-06,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,100.000000


In [37]:
pivot_df.isnull().sum() / len(pivot_df)

,0
Date,0.000000
environment_negative_percentage,0.566787
government_negative_percentage,0.386282
neutral_negative_percentage,0.801444
social_negative_percentage,0.158845
environment_positive_percentage,0.566787
government_positive_percentage,0.386282
neutral_positive_percentage,0.801444
social_positive_percentage,0.158845


In [38]:
pfizer_final = pd.merge(df_pfizer, pivot_df, on='Date')

In [39]:
pfizer_final.head(30)

,Date,Open,High,Low,Close,Adj Close,Volume,environment_negative_percentage,government_negative_percentage,neutral_negative_percentage,social_negative_percentage,environment_positive_percentage,government_positive_percentage,neutral_positive_percentage,social_positive_percentage
0,2023-08-28,36.439999,36.520000,36.029999,36.209999,34.193111,15101900,NaN,100.0,NaN,100.000000,NaN,0.0,NaN,0.000000
1,2023-08-29,36.259998,36.939999,36.000000,36.150002,34.136456,21076500,0.000000,0.0,NaN,NaN,100.000000,100.0,NaN,NaN
2,2023-08-30,36.180000,36.250000,35.820000,35.900002,33.900383,16833000,100.000000,NaN,100.0,0.000000,0.000000,NaN,0.0,100.000000
3,2023-08-31,35.939999,35.990002,35.380001,35.380001,33.409348,27919500,100.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
4,2023-09-01,35.639999,36.060001,35.599998,35.779999,33.787060,17519900,66.666667,NaN,NaN,NaN,33.333333,NaN,NaN,NaN
5,2023-09-05,35.959999,36.290001,35.349998,35.380001,33.409348,25856100,0.000000,NaN,NaN,0.000000,100.000000,NaN,NaN,100.000000
6,2023-09-06,35.259998,35.279999,34.259998,34.369999,32.455601,35370200,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,100.000000
7,2023-09-07,34.470001,34.639999,34.040001,34.279999,32.370613,26777700,0.000000,NaN,0.0,12.500000,100.000000,NaN,100.0,87.500000
8,2023-09-08,34.230000,34.389999,34.049999,34.250000,32.342285,19467700,0.000000,NaN,NaN,NaN,100.000000,NaN,NaN,NaN
9,2023-09-12,33.970001,34.209999,33.750000,34.150002,32.247856,17987200,NaN,100.0,NaN,100.000000,NaN,0.0,NaN,0.000000


In [40]:
pfizer_final.to_csv("/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Code] Models/[Dataset] Generated datasets/PFIZER-final.csv")

##Procter stock prices and news dataset

In [ ]:
data_procter = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Datasets]/[Dataset] Stock prices/PG.csv')
data_procter.head(20)

# Convert the 'Date' column in data_procter to datetime format
data_procter['Date'] = pd.to_datetime(data_procter['Date'], format='%Y-%m-%d').dt.date
procter_news = df[df['company'] == 'Procter']

merged_df_procter = pd.merge(procter_news, data_procter, on=['Date'])

merged_df_procter = merged_df_procter.drop(columns=['Volume', 'Adj Close'])

merged_df_procter.head()

,company,title,link,publish_date,label,type,Date,Open,High,Low,Close
0,Procter,Total number of employees of Procter & Gamble ...,https://news.google.com/read/CBMiowFBVV95cUxOQ...,2024-08-14 07:00:00+00:00,POSITIVE,social,2024-08-14,166.500000,169.470001,165.820007,168.800003
1,Procter,Net sales of Procter & Gamble in the U.S. 2014...,https://news.google.com/read/CBMijgFBVV95cUxON...,2024-08-14 07:00:00+00:00,POSITIVE,government,2024-08-14,166.500000,169.470001,165.820007,168.800003
2,Procter,The Procter & Gamble Company (NYSE:PG) Stake R...,https://news.google.com/read/CBMifEFVX3lxTE9ON...,2024-08-14 02:13:20+00:00,NEGATIVE,social,2024-08-14,166.500000,169.470001,165.820007,168.800003
3,Procter,Bank Pictet & Cie Asia Ltd. Sells 906 Shares o...,https://news.google.com/read/CBMiwAFBVV95cUxQd...,2024-08-14 06:46:43+00:00,NEGATIVE,social,2024-08-14,166.500000,169.470001,165.820007,168.800003
4,Procter,Procter & Gamble Company (The) (PG) is Attract...,https://news.google.com/read/CBMiigFBVV95cUxNT...,2024-08-16 13:00:13+00:00,POSITIVE,social,2024-08-16,167.820007,168.350006,166.800003,167.889999


In [ ]:
from collections import Counter

# Group by 'date' and 'type', aggregate sentiment scores and count news types
grouped_data_procter = procter_news.groupby(['Date', 'type']).agg({
    'label': lambda x: Counter(x),
    'type': 'count'
}).rename(columns={'type': 'type_count'}).reset_index()

print(grouped_data_procter)

          Date        type                           label  type_count
0   2024-01-19  government                 {'NEGATIVE': 1}           1
1   2024-01-23  government                 {'POSITIVE': 1}           1
2   2024-01-23      social                 {'POSITIVE': 1}           1
3   2024-01-31      social                 {'NEGATIVE': 1}           1
4   2024-02-01      social                 {'NEGATIVE': 3}           3
..         ...         ...                             ...         ...
71  2024-08-14      social  {'POSITIVE': 1, 'NEGATIVE': 2}           3
72  2024-08-15  government                 {'NEGATIVE': 1}           1
73  2024-08-16      social  {'POSITIVE': 2, 'NEGATIVE': 1}           3
74  2024-08-17      social  {'POSITIVE': 1, 'NEGATIVE': 3}           4
75  2024-08-18      social  {'POSITIVE': 1, 'NEGATIVE': 1}           2

[76 rows x 4 columns]


In [ ]:
# Initialize new columns for percentages
grouped_data_procter['negative_percentage'] = 0.0
grouped_data_procter['positive_percentage'] = 0.0

# Iterate through the rows to calculate the percentages
for i, row in grouped_data_procter.iterrows():
    sentiment_dict = row['label']

    # Get the counts of NEGATIVE and POSITIVE sentiments
    negative_count = sentiment_dict.get('NEGATIVE', 0)
    positive_count = sentiment_dict.get('POSITIVE', 0)
    total_count = row['type_count']

    # Calculate the percentages and assign to the new columns
    if total_count > 0:
        grouped_data_procter.at[i, 'negative_percentage'] = (negative_count / total_count) * 100
        grouped_data_procter.at[i, 'positive_percentage'] = (positive_count / total_count) * 100

# Display the updated DataFrame with the new columns
print(grouped_data_procter[['Date', 'type', 'negative_percentage', 'positive_percentage']])

          Date        type  negative_percentage  positive_percentage
0   2024-01-19  government           100.000000             0.000000
1   2024-01-23  government             0.000000           100.000000
2   2024-01-23      social             0.000000           100.000000
3   2024-01-31      social           100.000000             0.000000
4   2024-02-01      social           100.000000             0.000000
..         ...         ...                  ...                  ...
71  2024-08-14      social            66.666667            33.333333
72  2024-08-15  government           100.000000             0.000000
73  2024-08-16      social            33.333333            66.666667
74  2024-08-17      social            75.000000            25.000000
75  2024-08-18      social            50.000000            50.000000

[76 rows x 4 columns]


In [ ]:
# Pivot the DataFrame to get each type as separate columns for negative and positive percentages
pivot_df_procter = grouped_data_procter.pivot(index='Date', columns='type', values=['negative_percentage', 'positive_percentage'])

# Flatten the multi-level column names
pivot_df_procter.columns = [f"{col[1]}_{col[0]}" for col in pivot_df_procter.columns]

# Reset the index to bring 'Date' back as a column
pivot_df_procter = pivot_df_procter.reset_index()

# Display the resulting DataFrame
print(pivot_df_procter.head())

procter_final = pd.merge(data_procter, pivot_df_procter, on=['Date'])
procter_final.to_csv("/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Code] Models/[Dataset] Generated datasets/PROCTER-final.csv", index=False)

         Date  environment_negative_percentage  \
0  2024-01-19                              NaN   
1  2024-01-23                              NaN   
2  2024-01-31                              NaN   
3  2024-02-01                              NaN   
4  2024-02-02                              NaN   

   government_negative_percentage  neutral_negative_percentage  \
0                           100.0                          NaN   
1                             0.0                          NaN   
2                             NaN                          NaN   
3                             NaN                          NaN   
4                             NaN                        100.0   

   social_negative_percentage  environment_positive_percentage  \
0                         NaN                              NaN   
1                         0.0                              NaN   
2                       100.0                              NaN   
3                       100.0         

##Tencent stock prices and news dataset

In [49]:
data_tencent = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Datasets]/[Dataset] Stock prices/0700.HK.csv')
data_tencent.head(20)

# Convert the 'Date' column in data_tencent to datetime format
data_tencent['Date'] = pd.to_datetime(data_tencent['Date'], format='%Y-%m-%d').dt.date
tencent_news = df[df['company'] == 'Tencent']

merged_df_tencent = pd.merge(tencent_news, data_tencent, on=['Date'])

merged_df_tencent = merged_df_tencent.drop(columns=['Volume', 'Adj Close'])

merged_df_tencent.head()

,company,title,link,publish_date,label,type,Date,Open,High,Low,Close
0,Tencent,Tencent's second-quarter revenue rises on gami...,https://news.google.com/read/CBMimgFBVV95cUxOV...,2024-08-14 15:01:00+00:00,POSITIVE,social,2024-08-14,378.600006,379.399994,369.399994,373.799988
1,Tencent,Tencent’s Earnings Beat Fuels Hope of a China ...,https://news.google.com/read/CBMisgFBVV95cUxPM...,2024-08-14 13:56:32+00:00,POSITIVE,social,2024-08-14,378.600006,379.399994,369.399994,373.799988
2,Tencent,More than 1B Tencent user accounts leaked,https://news.google.com/read/CBMif0FVX3lxTE44a...,2024-08-14 13:12:00+00:00,NEGATIVE,social,2024-08-14,378.600006,379.399994,369.399994,373.799988
3,Tencent,Tencent (TCEHY) Q2 2024 Earnings Call Transcript,https://news.google.com/read/CBMiqAFBVV95cUxQY...,2024-08-14 18:45:17+00:00,NEGATIVE,social,2024-08-14,378.600006,379.399994,369.399994,373.799988
4,Tencent,Chinese tech giant Tencent's quarterly profit ...,https://news.google.com/read/CBMibkFVX3lxTE02c...,2024-08-14 08:38:01+00:00,POSITIVE,social,2024-08-14,378.600006,379.399994,369.399994,373.799988


In [50]:
from collections import Counter

# Group by 'date' and 'type', aggregate sentiment scores and count news types
grouped_data_tencent = tencent_news.groupby(['Date', 'type']).agg({
    'label': lambda x: Counter(x),
    'type': 'count'
}).rename(columns={'type': 'type_count'}).reset_index()

print(grouped_data_tencent)

          Date        type                           label  type_count
0   2023-09-03      social                 {'NEGATIVE': 1}           1
1   2023-09-06      social                 {'POSITIVE': 1}           1
2   2023-09-07      social                 {'POSITIVE': 2}           2
3   2023-10-19  government                 {'POSITIVE': 1}           1
4   2023-11-10  government                 {'NEGATIVE': 1}           1
..         ...         ...                             ...         ...
71  2024-07-29     neutral                 {'NEGATIVE': 1}           1
72  2024-08-13      social  {'POSITIVE': 1, 'NEGATIVE': 1}           2
73  2024-08-14      social  {'POSITIVE': 3, 'NEGATIVE': 2}           5
74  2024-08-15      social                 {'POSITIVE': 1}           1
75  2024-08-20      social                 {'POSITIVE': 1}           1

[76 rows x 4 columns]


In [51]:
# Initialize new columns for percentages
grouped_data_tencent['negative_percentage'] = 0.0
grouped_data_tencent['positive_percentage'] = 0.0

# Iterate through the rows to calculate the percentages
for i, row in grouped_data_tencent.iterrows():
    sentiment_dict = row['label']

    # Get the counts of NEGATIVE and POSITIVE sentiments
    negative_count = sentiment_dict.get('NEGATIVE', 0)
    positive_count = sentiment_dict.get('POSITIVE', 0)
    total_count = row['type_count']

    # Calculate the percentages and assign to the new columns
    if total_count > 0:
        grouped_data_tencent.at[i, 'negative_percentage'] = (negative_count / total_count) * 100
        grouped_data_tencent.at[i, 'positive_percentage'] = (positive_count / total_count) * 100

# Display the updated DataFrame with the new columns
print(grouped_data_tencent[['Date', 'type', 'negative_percentage', 'positive_percentage']])

          Date        type  negative_percentage  positive_percentage
0   2023-09-03      social                100.0                  0.0
1   2023-09-06      social                  0.0                100.0
2   2023-09-07      social                  0.0                100.0
3   2023-10-19  government                  0.0                100.0
4   2023-11-10  government                100.0                  0.0
..         ...         ...                  ...                  ...
71  2024-07-29     neutral                100.0                  0.0
72  2024-08-13      social                 50.0                 50.0
73  2024-08-14      social                 40.0                 60.0
74  2024-08-15      social                  0.0                100.0
75  2024-08-20      social                  0.0                100.0

[76 rows x 4 columns]


In [52]:
# Pivot the DataFrame to get each type as separate columns for negative and positive percentages
pivot_df_tencent = grouped_data_tencent.pivot(index='Date', columns='type', values=['negative_percentage', 'positive_percentage'])

# Flatten the multi-level column names
pivot_df_tencent.columns = [f"{col[1]}_{col[0]}" for col in pivot_df_tencent.columns]

# Reset the index to bring 'Date' back as a column
pivot_df_tencent = pivot_df_tencent.reset_index()

# Display the resulting DataFrame
print(pivot_df_tencent.head())

tencent_final = pd.merge(data_tencent, pivot_df_tencent, on=['Date'])
tencent_final.to_csv("/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Code] Models/[Dataset] Generated datasets/TENCENT-final.csv", index=False)

         Date  environment_negative_percentage  \
0  2023-09-03                              NaN   
1  2023-09-06                              NaN   
2  2023-09-07                              NaN   
3  2023-10-19                              NaN   
4  2023-11-10                              NaN   

   government_negative_percentage  neutral_negative_percentage  \
0                             NaN                          NaN   
1                             NaN                          NaN   
2                             NaN                          NaN   
3                             0.0                          NaN   
4                           100.0                          NaN   

   social_negative_percentage  environment_positive_percentage  \
0                       100.0                              NaN   
1                         0.0                              NaN   
2                         0.0                              NaN   
3                         NaN         

##Samsung stock prices and news dataset

In [44]:
data_samsung = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Datasets]/[Dataset] Stock prices/005930.KS.csv')
data_samsung.head(20)

# Convert the 'Date' column in data_samsung to datetime format
data_samsung['Date'] = pd.to_datetime(data_samsung['Date'], format='%Y-%m-%d').dt.date
samsung_news = df[df['company'] == 'Samsung']

merged_df_samsung = pd.merge(samsung_news, data_samsung, on=['Date'])

merged_df_samsung = merged_df_samsung.drop(columns=['Volume', 'Adj Close'])

merged_df_samsung.head()

,company,title,link,publish_date,label,type,Date,Open,High,Low,Close
0,Samsung,[Editorial] Solving the Digital Health Dilemma...,https://news.google.com/read/CBMixgFBVV95cUxQM...,2024-02-26 08:00:00+00:00,POSITIVE,social,2024-02-26,72300.0,73200.0,72200.0,72800.0
1,Samsung,Samsung Expands Circle to Search to Galaxy A S...,https://news.google.com/read/CBMisAFBVV95cUxNc...,2024-08-14 13:02:54+00:00,POSITIVE,social,2024-08-14,77400.0,77800.0,77000.0,77200.0
2,Samsung,Samsung Family Care app helps caregivers and a...,https://news.google.com/read/CBMilAFBVV95cUxPZ...,2024-08-14 13:11:56+00:00,POSITIVE,social,2024-08-14,77400.0,77800.0,77000.0,77200.0
3,Samsung,Samsung UK | Mobile | Home Electronics | Home ...,https://news.google.com/read/CBMiQEFVX3lxTE5yS...,2024-08-14 02:26:15+00:00,POSITIVE,social,2024-08-14,77400.0,77800.0,77000.0,77200.0
4,Samsung,One UI 6.1 Brings Galaxy AI From S24 Series to...,https://news.google.com/read/CBMipgFBVV95cUxOY...,2024-04-15 07:00:00+00:00,POSITIVE,government,2024-04-15,82900.0,83200.0,81200.0,82200.0


In [45]:
from collections import Counter

# Group by 'date' and 'type', aggregate sentiment scores and count news types
grouped_data_samsung = samsung_news.groupby(['Date', 'type']).agg({
    'label': lambda x: Counter(x),
    'type': 'count'
}).rename(columns={'type': 'type_count'}).reset_index()

print(grouped_data_samsung)

          Date         type            label  type_count
0   2023-08-29  environment  {'POSITIVE': 1}           1
1   2023-08-30       social  {'POSITIVE': 1}           1
2   2023-09-07       social  {'POSITIVE': 1}           1
3   2023-09-15       social  {'NEGATIVE': 1}           1
4   2023-09-20   government  {'POSITIVE': 2}           2
..         ...          ...              ...         ...
80  2024-07-27       social  {'NEGATIVE': 1}           1
81  2024-08-08  environment  {'NEGATIVE': 1}           1
82  2024-08-11       social  {'NEGATIVE': 1}           1
83  2024-08-14       social  {'POSITIVE': 3}           3
84  2024-08-18       social  {'POSITIVE': 1}           1

[85 rows x 4 columns]


In [46]:
# Initialize new columns for percentages
grouped_data_samsung['negative_percentage'] = 0.0
grouped_data_samsung['positive_percentage'] = 0.0

# Iterate through the rows to calculate the percentages
for i, row in grouped_data_samsung.iterrows():
    sentiment_dict = row['label']

    # Get the counts of NEGATIVE and POSITIVE sentiments
    negative_count = sentiment_dict.get('NEGATIVE', 0)
    positive_count = sentiment_dict.get('POSITIVE', 0)
    total_count = row['type_count']

    # Calculate the percentages and assign to the new columns
    if total_count > 0:
        grouped_data_samsung.at[i, 'negative_percentage'] = (negative_count / total_count) * 100
        grouped_data_samsung.at[i, 'positive_percentage'] = (positive_count / total_count) * 100

# Display the updated DataFrame with the new columns
print(grouped_data_samsung[['Date', 'type', 'negative_percentage', 'positive_percentage']])

          Date         type  negative_percentage  positive_percentage
0   2023-08-29  environment                  0.0                100.0
1   2023-08-30       social                  0.0                100.0
2   2023-09-07       social                  0.0                100.0
3   2023-09-15       social                100.0                  0.0
4   2023-09-20   government                  0.0                100.0
..         ...          ...                  ...                  ...
80  2024-07-27       social                100.0                  0.0
81  2024-08-08  environment                100.0                  0.0
82  2024-08-11       social                100.0                  0.0
83  2024-08-14       social                  0.0                100.0
84  2024-08-18       social                  0.0                100.0

[85 rows x 4 columns]


In [48]:
# Pivot the DataFrame to get each type as separate columns for negative and positive percentages
pivot_df_samsung = grouped_data_samsung.pivot(index='Date', columns='type', values=['negative_percentage', 'positive_percentage'])

# Flatten the multi-level column names
pivot_df_samsung.columns = [f"{col[1]}_{col[0]}" for col in pivot_df_samsung.columns]

# Reset the index to bring 'Date' back as a column
pivot_df_samsung = pivot_df_samsung.reset_index()

# Display the resulting DataFrame
print(pivot_df_samsung.head())

samsung_final = pd.merge(data_samsung, pivot_df_samsung, on=['Date'])
samsung_final.to_csv("/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Code] Models/[Dataset] Generated datasets/SAMSUNG-final.csv", index=False)

         Date  environment_negative_percentage  \
0  2023-08-29                              0.0   
1  2023-08-30                              NaN   
2  2023-09-07                              NaN   
3  2023-09-15                              NaN   
4  2023-09-20                              NaN   

   government_negative_percentage  neutral_negative_percentage  \
0                             NaN                          NaN   
1                             NaN                          NaN   
2                             NaN                          NaN   
3                             NaN                          NaN   
4                             0.0                          NaN   

   social_negative_percentage  environment_positive_percentage  \
0                         NaN                            100.0   
1                         0.0                              NaN   
2                         0.0                              NaN   
3                       100.0         

##Tesla stock prices and news dataset

In [53]:
df['company'].unique()

array(['Oracle', 'Procter', 'Pfizer', 'Samsung', 'Shell', 'Tencent',
       'Toyota', 'Tesla', 'Unilever', 'UnitedHealth', 'Visa', 'Walmart',
       'Intel'], dtype=object)

In [54]:
data_tesla = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Datasets]/[Dataset] Stock prices/TSLA.csv')
data_tesla.head(20)

# Convert the 'Date' column in data_tesla to datetime format
data_tesla['Date'] = pd.to_datetime(data_tesla['Date'], format='%Y-%m-%d').dt.date
tesla_news = df[df['company'] == 'Tesla']

merged_df_tesla = pd.merge(tesla_news, data_tesla, on=['Date'])

merged_df_tesla = merged_df_tesla.drop(columns=['Volume', 'Adj Close'])

merged_df_tesla.head()

,company,title,link,publish_date,label,type,Date,Open,High,Low,Close
0,Tesla,Tesla sales plunge far more than expected,https://news.google.com/read/CBMib0FVX3lxTE9MR...,2024-04-02 07:00:00+00:00,NEGATIVE,social,2024-04-02,164.750000,167.690002,163.429993,166.630005
1,Tesla,Tesla shares fall after deliveries drop 8.5% f...,https://news.google.com/read/CBMimwFBVV95cUxPZ...,2024-04-02 07:00:00+00:00,NEGATIVE,environment,2024-04-02,164.750000,167.690002,163.429993,166.630005
2,Tesla,Tesla quarterly deliveries decline for the fir...,https://news.google.com/read/CBMitgFBVV95cUxOU...,2024-04-02 07:00:00+00:00,NEGATIVE,environment,2024-04-02,164.750000,167.690002,163.429993,166.630005
3,Tesla,"Tesla’s Sales Drop, a Sign That Its Grip on th...",https://news.google.com/read/CBMihgFBVV95cUxOe...,2024-04-02 07:00:00+00:00,NEGATIVE,environment,2024-04-02,164.750000,167.690002,163.429993,166.630005
4,Tesla,"Tesla, Volvo Car pause output as Red Sea shipp...",https://news.google.com/read/CBMi0gFBVV95cUxNX...,2024-01-12 08:00:00+00:00,NEGATIVE,social,2024-01-12,220.080002,225.339996,217.149994,218.889999


In [55]:
# Group by 'date' and 'type', aggregate sentiment scores and count news types
grouped_data_tesla = tesla_news.groupby(['Date', 'type']).agg({
    'label': lambda x: Counter(x),
    'type': 'count'
}).rename(columns={'type': 'type_count'}).reset_index()

print(grouped_data_tesla)

          Date         type            label  type_count
0   2023-08-28       social  {'NEGATIVE': 1}           1
1   2023-08-30      neutral  {'NEGATIVE': 1}           1
2   2023-08-30       social  {'POSITIVE': 1}           1
3   2023-09-01  environment  {'POSITIVE': 1}           1
4   2023-09-07       social  {'POSITIVE': 1}           1
..         ...          ...              ...         ...
77  2024-08-15       social  {'NEGATIVE': 1}           1
78  2024-08-16       social  {'NEGATIVE': 1}           1
79  2024-08-18       social  {'NEGATIVE': 1}           1
80  2024-08-19  environment  {'NEGATIVE': 1}           1
81  2024-08-19       social  {'NEGATIVE': 1}           1

[82 rows x 4 columns]


In [56]:
# Initialize new columns for percentages
grouped_data_tesla['negative_percentage'] = 0.0
grouped_data_tesla['positive_percentage'] = 0.0

# Iterate through the rows to calculate the percentages
for i, row in grouped_data_tesla.iterrows():
    sentiment_dict = row['label']

    # Get the counts of NEGATIVE and POSITIVE sentiments
    negative_count = sentiment_dict.get('NEGATIVE', 0)
    positive_count = sentiment_dict.get('POSITIVE', 0)
    total_count = row['type_count']

    # Calculate the percentages and assign to the new columns
    if total_count > 0:
        grouped_data_tesla.at[i, 'negative_percentage'] = (negative_count / total_count) * 100
        grouped_data_tesla.at[i, 'positive_percentage'] = (positive_count / total_count) * 100

# Display the updated DataFrame with the new columns
print(grouped_data_tesla[['Date', 'type', 'negative_percentage', 'positive_percentage']])

          Date         type  negative_percentage  positive_percentage
0   2023-08-28       social                100.0                  0.0
1   2023-08-30      neutral                100.0                  0.0
2   2023-08-30       social                  0.0                100.0
3   2023-09-01  environment                  0.0                100.0
4   2023-09-07       social                  0.0                100.0
..         ...          ...                  ...                  ...
77  2024-08-15       social                100.0                  0.0
78  2024-08-16       social                100.0                  0.0
79  2024-08-18       social                100.0                  0.0
80  2024-08-19  environment                100.0                  0.0
81  2024-08-19       social                100.0                  0.0

[82 rows x 4 columns]


In [57]:
# Pivot the DataFrame to get each type as separate columns for negative and positive percentages
pivot_df_tesla = grouped_data_tesla.pivot(index='Date', columns='type', values=['negative_percentage', 'positive_percentage'])

# Flatten the multi-level column names
pivot_df_tesla.columns = [f"{col[1]}_{col[0]}" for col in pivot_df_tesla.columns]

# Reset the index to bring 'Date' back as a column
pivot_df_tesla = pivot_df_tesla.reset_index()

# Display the resulting DataFrame
print(pivot_df_tesla.head())

tesla_final = pd.merge(data_tesla, pivot_df_tesla, on=['Date'])
tesla_final.to_csv("/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Code] Models/[Dataset] Generated datasets/TESLA-final.csv", index=False)

         Date  environment_negative_percentage  \
0  2023-08-28                              NaN   
1  2023-08-30                              NaN   
2  2023-09-01                              0.0   
3  2023-09-07                              NaN   
4  2023-09-08                              0.0   

   government_negative_percentage  neutral_negative_percentage  \
0                             NaN                          NaN   
1                             NaN                        100.0   
2                             NaN                          NaN   
3                             NaN                          NaN   
4                             NaN                          NaN   

   social_negative_percentage  environment_positive_percentage  \
0                       100.0                              NaN   
1                         0.0                              NaN   
2                         NaN                            100.0   
3                         0.0         

##Visa stock prices and news dataset

In [58]:
data_visa = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Datasets]/[Dataset] Stock prices/V.csv')
data_visa.head(20)

# Convert the 'Date' column in data_visa to datetime format
data_visa['Date'] = pd.to_datetime(data_visa['Date'], format='%Y-%m-%d').dt.date
visa_news = df[df['company'] == 'Visa']

merged_df_visa = pd.merge(visa_news, data_visa, on=['Date'])

merged_df_visa = merged_df_visa.drop(columns=['Volume', 'Adj Close'])

merged_df_visa.head()

,company,title,link,publish_date,label,type,Date,Open,High,Low,Close
0,Visa,Visa Everywhere Initiative 2024,https://news.google.com/read/CBMiiwFBVV95cUxPb...,2024-03-22 01:32:05+00:00,POSITIVE,government,2024-03-22,290.000000,290.489990,282.890015,283.260010
1,Visa,Pursuing Visa Restrictions to Address Repressi...,https://news.google.com/read/CBMi0AFBVV95cUxQR...,2024-07-12 07:00:00+00:00,NEGATIVE,social,2024-07-12,264.339996,267.399994,263.660004,265.739990
2,Visa,Visa and Inter Miami CF Announce International...,https://news.google.com/read/CBMilwFBVV95cUxQU...,2024-05-02 07:00:00+00:00,POSITIVE,government,2024-05-02,269.350006,269.470001,266.649994,267.609985
3,Visa,Electronic System for Travel Authorization,https://news.google.com/read/CBMiZEFVX3lxTE5iM...,2024-03-28 07:00:00+00:00,POSITIVE,government,2024-03-28,278.489990,279.799988,277.119995,279.079987
4,Visa,State Department Testing Digital Visa Authoriz...,https://news.google.com/read/CBMi0wFBVV95cUxNe...,2023-09-19 07:00:00+00:00,NEGATIVE,government,2023-09-19,245.000000,245.119995,241.460007,243.559998


In [59]:
# Group by 'date' and 'type', aggregate sentiment scores and count news types
grouped_data_visa = visa_news.groupby(['Date', 'type']).agg({
    'label': lambda x: Counter(x),
    'type': 'count'
}).rename(columns={'type': 'type_count'}).reset_index()

print(grouped_data_visa)

          Date        type            label  type_count
0   2023-08-22  government  {'NEGATIVE': 1}           1
1   2023-08-24  government  {'NEGATIVE': 1}           1
2   2023-09-06      social  {'POSITIVE': 1}           1
3   2023-09-13  government  {'POSITIVE': 1}           1
4   2023-09-18  government  {'NEGATIVE': 1}           1
..         ...         ...              ...         ...
81  2024-07-12      social  {'NEGATIVE': 1}           1
82  2024-08-06  government  {'POSITIVE': 1}           1
83  2024-08-08  government  {'NEGATIVE': 1}           1
84  2024-08-12      social  {'POSITIVE': 1}           1
85  2024-08-14  government  {'NEGATIVE': 1}           1

[86 rows x 4 columns]


In [60]:
# Initialize new columns for percentages
grouped_data_visa['negative_percentage'] = 0.0
grouped_data_visa['positive_percentage'] = 0.0

# Iterate through the rows to calculate the percentages
for i, row in grouped_data_visa.iterrows():
    sentiment_dict = row['label']

    # Get the counts of NEGATIVE and POSITIVE sentiments
    negative_count = sentiment_dict.get('NEGATIVE', 0)
    positive_count = sentiment_dict.get('POSITIVE', 0)
    total_count = row['type_count']

    # Calculate the percentages and assign to the new columns
    if total_count > 0:
        grouped_data_visa.at[i, 'negative_percentage'] = (negative_count / total_count) * 100
        grouped_data_visa.at[i, 'positive_percentage'] = (positive_count / total_count) * 100

# Display the updated DataFrame with the new columns
print(grouped_data_visa[['Date', 'type', 'negative_percentage', 'positive_percentage']])

          Date        type  negative_percentage  positive_percentage
0   2023-08-22  government                100.0                  0.0
1   2023-08-24  government                100.0                  0.0
2   2023-09-06      social                  0.0                100.0
3   2023-09-13  government                  0.0                100.0
4   2023-09-18  government                100.0                  0.0
..         ...         ...                  ...                  ...
81  2024-07-12      social                100.0                  0.0
82  2024-08-06  government                  0.0                100.0
83  2024-08-08  government                100.0                  0.0
84  2024-08-12      social                  0.0                100.0
85  2024-08-14  government                100.0                  0.0

[86 rows x 4 columns]


In [61]:
# Pivot the DataFrame to get each type as separate columns for negative and positive percentages
pivot_df_visa = grouped_data_visa.pivot(index='Date', columns='type', values=['negative_percentage', 'positive_percentage'])

# Flatten the multi-level column names
pivot_df_visa.columns = [f"{col[1]}_{col[0]}" for col in pivot_df_visa.columns]

# Reset the index to bring 'Date' back as a column
pivot_df_visa = pivot_df_visa.reset_index()

# Display the resulting DataFrame
print(pivot_df_visa.head())

visa_final = pd.merge(data_visa, pivot_df_visa, on=['Date'])
visa_final.to_csv("/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Code] Models/[Dataset] Generated datasets/VISA-final.csv", index=False)

         Date  environment_negative_percentage  \
0  2023-08-22                              NaN   
1  2023-08-24                              NaN   
2  2023-09-06                              NaN   
3  2023-09-13                              NaN   
4  2023-09-18                              NaN   

   government_negative_percentage  neutral_negative_percentage  \
0                           100.0                          NaN   
1                           100.0                          NaN   
2                             NaN                          NaN   
3                             0.0                          NaN   
4                           100.0                          NaN   

   social_negative_percentage  environment_positive_percentage  \
0                         NaN                              NaN   
1                         NaN                              NaN   
2                         0.0                              NaN   
3                         NaN         

##Intel stock prices and news dataset

In [63]:
data_intel = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Datasets]/[Dataset] Stock prices/INTC.csv')
data_intel.head(20)

# Convert the 'Date' column in data_intel to datetime format
data_intel['Date'] = pd.to_datetime(data_intel['Date'], format='%Y-%m-%d').dt.date
intel_news = df[df['company'] == 'Intel']

merged_df_intel = pd.merge(intel_news, data_intel, on=['Date'])

merged_df_intel = merged_df_intel.drop(columns=['Volume', 'Adj Close'])

merged_df_intel.head()

,company,title,link,publish_date,label,type,Date,Open,High,Low,Close
0,Intel,Intel Accelerates AI Everywhere with Launch of...,https://news.google.com/read/CBMitgFBVV95cUxNN...,2023-12-14 08:00:00+00:00,POSITIVE,neutral,2023-12-14,45.009998,47.07,44.650002,45.180000
1,Intel,Intel’s Core Ultra CPUs are here — and they al...,https://news.google.com/read/CBMijAFBVV95cUxOV...,2023-12-14 08:00:00+00:00,NEGATIVE,social,2023-12-14,45.009998,47.07,44.650002,45.180000
2,Intel,Intel unveils new AI chip to compete with Nvid...,https://news.google.com/read/CBMinAFBVV95cUxQO...,2023-12-14 08:00:00+00:00,POSITIVE,social,2023-12-14,45.009998,47.07,44.650002,45.180000
3,Intel,Using an Intel Core Ultra laptop for anything ...,https://news.google.com/read/CBMi3gFBVV95cUxOV...,2023-12-14 08:00:00+00:00,NEGATIVE,neutral,2023-12-14,45.009998,47.07,44.650002,45.180000
4,Intel,Intel’s Humbling,https://news.google.com/read/CBMiWEFVX3lxTFA4Z...,2024-01-30 08:00:00+00:00,NEGATIVE,social,2024-01-30,43.669998,43.82,42.889999,42.919998


In [64]:
# Group by 'date' and 'type', aggregate sentiment scores and count news types
grouped_data_intel = intel_news.groupby(['Date', 'type']).agg({
    'label': lambda x: Counter(x),
    'type': 'count'
}).rename(columns={'type': 'type_count'}).reset_index()

print(grouped_data_intel)

          Date         type                           label  type_count
0   2023-09-12       social                 {'NEGATIVE': 1}           1
1   2023-09-19  environment                 {'POSITIVE': 1}           1
2   2023-09-19       social                 {'POSITIVE': 2}           2
3   2023-11-06   government                 {'NEGATIVE': 1}           1
4   2023-11-07  environment                 {'POSITIVE': 1}           1
5   2023-12-14      neutral  {'POSITIVE': 1, 'NEGATIVE': 1}           2
6   2023-12-14       social  {'NEGATIVE': 1, 'POSITIVE': 1}           2
7   2023-12-15       social                 {'POSITIVE': 1}           1
8   2024-01-03   government                 {'POSITIVE': 1}           1
9   2024-01-08  environment                 {'POSITIVE': 1}           1
10  2024-01-09       social                 {'NEGATIVE': 1}           1
11  2024-01-18       social  {'POSITIVE': 1, 'NEGATIVE': 1}           2
12  2024-01-22       social                 {'POSITIVE': 1}     

In [65]:
# Initialize new columns for percentages
grouped_data_intel['negative_percentage'] = 0.0
grouped_data_intel['positive_percentage'] = 0.0

# Iterate through the rows to calculate the percentages
for i, row in grouped_data_intel.iterrows():
    sentiment_dict = row['label']

    # Get the counts of NEGATIVE and POSITIVE sentiments
    negative_count = sentiment_dict.get('NEGATIVE', 0)
    positive_count = sentiment_dict.get('POSITIVE', 0)
    total_count = row['type_count']

    # Calculate the percentages and assign to the new columns
    if total_count > 0:
        grouped_data_intel.at[i, 'negative_percentage'] = (negative_count / total_count) * 100
        grouped_data_intel.at[i, 'positive_percentage'] = (positive_count / total_count) * 100

# Display the updated DataFrame with the new columns
print(grouped_data_intel[['Date', 'type', 'negative_percentage', 'positive_percentage']])

          Date         type  negative_percentage  positive_percentage
0   2023-09-12       social           100.000000             0.000000
1   2023-09-19  environment             0.000000           100.000000
2   2023-09-19       social             0.000000           100.000000
3   2023-11-06   government           100.000000             0.000000
4   2023-11-07  environment             0.000000           100.000000
5   2023-12-14      neutral            50.000000            50.000000
6   2023-12-14       social            50.000000            50.000000
7   2023-12-15       social             0.000000           100.000000
8   2024-01-03   government             0.000000           100.000000
9   2024-01-08  environment             0.000000           100.000000
10  2024-01-09       social           100.000000             0.000000
11  2024-01-18       social            50.000000            50.000000
12  2024-01-22       social             0.000000           100.000000
13  2024-01-24   gov

In [66]:
# Pivot the DataFrame to get each type as separate columns for negative and positive percentages
pivot_df_intel = grouped_data_intel.pivot(index='Date', columns='type', values=['negative_percentage', 'positive_percentage'])

# Flatten the multi-level column names
pivot_df_intel.columns = [f"{col[1]}_{col[0]}" for col in pivot_df_intel.columns]

# Reset the index to bring 'Date' back as a column
pivot_df_intel = pivot_df_intel.reset_index()

# Display the resulting DataFrame
print(pivot_df_intel.head())

intel_final = pd.merge(data_intel, pivot_df_intel, on=['Date'])
intel_final.to_csv("/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Code] Models/[Dataset] Generated datasets/INTEL-final.csv", index=False)

         Date  environment_negative_percentage  \
0  2023-09-12                              NaN   
1  2023-09-19                              0.0   
2  2023-11-06                              NaN   
3  2023-11-07                              0.0   
4  2023-12-14                              NaN   

   government_negative_percentage  neutral_negative_percentage  \
0                             NaN                          NaN   
1                             NaN                          NaN   
2                           100.0                          NaN   
3                             NaN                          NaN   
4                             NaN                         50.0   

   social_negative_percentage  environment_positive_percentage  \
0                       100.0                              NaN   
1                         0.0                            100.0   
2                         NaN                              NaN   
3                         NaN         

##Walmart stock prices and news dataset

In [20]:
data_walmart = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Datasets]/[Dataset] Stock prices/WMT.csv')
data_walmart.head(20)

# Convert the 'Date' column in data_walmart to datetime format
data_walmart['Date'] = pd.to_datetime(data_walmart['Date'], format='%Y-%m-%d').dt.date
walmart_news = df[df['company'] == 'Walmart']

merged_df_walmart = pd.merge(walmart_news, data_walmart, on=['Date'])

merged_df_walmart = merged_df_walmart.drop(columns=['Volume', 'Adj Close'])

merged_df_walmart.head()

,company,title,link,publish_date,label,type,Date,Open,High,Low,Close
0,Walmart,Our partnership with Walmart brings big change,https://news.google.com/read/CBMid0FVX3lxTE4xd...,2024-06-05 07:00:00+00:00,POSITIVE,social,2024-06-05,66.570000,67.150002,66.279999,67.089996
1,Walmart,"Easter Savings Hop to Walmart, Featuring an Ea...",https://news.google.com/read/CBMiwAFBVV95cUxPW...,2024-03-04 08:00:00+00:00,NEGATIVE,social,2024-03-04,58.500000,59.389999,58.400002,59.299999
2,Walmart,Walmart unveils new generative AI-powered capa...,https://news.google.com/read/CBMixgFBVV95cUxQb...,2024-01-09 08:00:00+00:00,POSITIVE,social,2024-01-09,52.643333,53.113335,52.576668,53.103333
3,Walmart,Sky High Ambitions: Walmart To Make Largest Dr...,https://news.google.com/read/CBMizAFBVV95cUxPd...,2024-01-09 08:00:00+00:00,POSITIVE,social,2024-01-09,52.643333,53.113335,52.576668,53.103333
4,Walmart,Walmart Offers a Glimpse Into the Future of Re...,https://news.google.com/read/CBMiwwFBVV95cUxNc...,2024-01-09 08:00:00+00:00,POSITIVE,social,2024-01-09,52.643333,53.113335,52.576668,53.103333


In [22]:
from collections import Counter

# Group by 'date' and 'type', aggregate sentiment scores and count news types
grouped_data_walmart = walmart_news.groupby(['Date', 'type']).agg({
    'label': lambda x: Counter(x),
    'type': 'count'
}).rename(columns={'type': 'type_count'}).reset_index()

print(grouped_data_walmart)

          Date        type            label  type_count
0   2023-08-24      social  {'NEGATIVE': 1}           1
1   2023-09-07      social  {'POSITIVE': 2}           2
2   2023-09-19      social  {'POSITIVE': 1}           1
3   2023-09-22      social  {'POSITIVE': 1}           1
4   2023-09-26      social  {'POSITIVE': 1}           1
..         ...         ...              ...         ...
71  2024-07-11      social  {'POSITIVE': 1}           1
72  2024-07-18      social  {'NEGATIVE': 1}           1
73  2024-07-30      social  {'NEGATIVE': 1}           1
74  2024-08-08      social  {'POSITIVE': 1}           1
75  2024-08-15  government  {'NEGATIVE': 1}           1

[76 rows x 4 columns]


In [23]:
# Initialize new columns for percentages
grouped_data_walmart['negative_percentage'] = 0.0
grouped_data_walmart['positive_percentage'] = 0.0

# Iterate through the rows to calculate the percentages
for i, row in grouped_data_walmart.iterrows():
    sentiment_dict = row['label']

    # Get the counts of NEGATIVE and POSITIVE sentiments
    negative_count = sentiment_dict.get('NEGATIVE', 0)
    positive_count = sentiment_dict.get('POSITIVE', 0)
    total_count = row['type_count']

    # Calculate the percentages and assign to the new columns
    if total_count > 0:
        grouped_data_walmart.at[i, 'negative_percentage'] = (negative_count / total_count) * 100
        grouped_data_walmart.at[i, 'positive_percentage'] = (positive_count / total_count) * 100

# Display the updated DataFrame with the new columns
print(grouped_data_walmart[['Date', 'type', 'negative_percentage', 'positive_percentage']])

          Date        type  negative_percentage  positive_percentage
0   2023-08-24      social                100.0                  0.0
1   2023-09-07      social                  0.0                100.0
2   2023-09-19      social                  0.0                100.0
3   2023-09-22      social                  0.0                100.0
4   2023-09-26      social                  0.0                100.0
..         ...         ...                  ...                  ...
71  2024-07-11      social                  0.0                100.0
72  2024-07-18      social                100.0                  0.0
73  2024-07-30      social                100.0                  0.0
74  2024-08-08      social                  0.0                100.0
75  2024-08-15  government                100.0                  0.0

[76 rows x 4 columns]


In [24]:
# Pivot the DataFrame to get each type as separate columns for negative and positive percentages
pivot_df_walmart = grouped_data_walmart.pivot(index='Date', columns='type', values=['negative_percentage', 'positive_percentage'])

# Flatten the multi-level column names
pivot_df_walmart.columns = [f"{col[1]}_{col[0]}" for col in pivot_df_walmart.columns]

# Reset the index to bring 'Date' back as a column
pivot_df_walmart = pivot_df_walmart.reset_index()

# Display the resulting DataFrame
print(pivot_df_walmart.head())

intel_walmart = pd.merge(data_walmart, pivot_df_walmart, on=['Date'])
intel_walmart.to_csv("/content/drive/MyDrive/Colab Notebooks/[DS] Seminarska/[Code] Models/[Dataset] Generated datasets/WALMART-final.csv", index=False)

         Date  environment_negative_percentage  \
0  2023-08-24                              NaN   
1  2023-09-07                              NaN   
2  2023-09-19                              NaN   
3  2023-09-22                              NaN   
4  2023-09-26                              NaN   

   government_negative_percentage  neutral_negative_percentage  \
0                             NaN                          NaN   
1                             NaN                          NaN   
2                             NaN                          NaN   
3                             NaN                          NaN   
4                             NaN                          NaN   

   social_negative_percentage  environment_positive_percentage  \
0                       100.0                              NaN   
1                         0.0                              NaN   
2                         0.0                              NaN   
3                         0.0         